In [3]:
#libraries
import pandas as pd
import os
print(os.getcwd())

C:\Users\S P E C T R E\Desktop\Personal Projects\umpire


In [4]:
base_path = r"C:\Users\S P E C T R E\Desktop\Personal Projects\umpire\gtfs-accra-ghana-2016"

In [6]:
stop_times = pd.read_csv(f"{base_path}\\stop_times.csv")
trips = pd.read_csv(f"{base_path}\\trips.csv")
routes = pd.read_csv(f"{base_path}\\routes.csv")
calendar = pd.read_csv(f"{base_path}\\calendar.csv")
shapes = pd.read_csv(f"{base_path}\\shapes.csv")
stops = pd.read_csv(f"{base_path}\\stops.csv")
fare_rules = pd.read_csv(f"{base_path}\\fare_rules.csv")
fare_attributes = pd.read_csv(f"{base_path}\\fare_attributes.csv")
agency = pd.read_csv(f"{base_path}\\agency.csv")